In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, mutual_info_classif, VarianceThreshold
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

dataset = pd.read_csv('combined_data.csv')
features = dataset.drop(columns=['Frame', 'class'])
labels = dataset['class']

variance_filter = VarianceThreshold()
filtered_features = variance_filter.fit_transform(features)

train_features, test_features, train_labels, test_labels = train_test_split(
    filtered_features, labels, test_size=0.3, random_state=42)

pipeline_svm = make_pipeline(
    RobustScaler(),
    SVC()
)

param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf']
}

svm_grid_search = GridSearchCV(pipeline_svm, param_grid, cv=10)
svm_grid_search.fit(train_features, train_labels)

best_svm_model = svm_grid_search.best_estimator_
svm_test_accuracy = best_svm_model.score(test_features, test_labels)
print("SVM Accuracy: %.2f%%" % (svm_test_accuracy * 100))
print("Best SVM Parameters:", svm_grid_search.best_params_)
print("Best SVM Cross-Validation Accuracy: %.2f%%" % (svm_grid_search.best_score_ * 100))

k_best_selector = SelectKBest(mutual_info_classif, k=10)
pipeline_svm_kbest = make_pipeline(
    k_best_selector,
    SVC()
)

svm_grid_search_kbest = GridSearchCV(pipeline_svm_kbest, param_grid, cv=10)
svm_grid_search_kbest.fit(train_features, train_labels)
print("Best SVM with Feature Selection Parameters:", svm_grid_search_kbest.best_params_)
print("Best SVM with Feature Selection Cross-Validation Accuracy: %.2f%%" % (svm_grid_search_kbest.best_score_ * 100))

pca_transformer = PCA(n_components=10)
pipeline_svm_pca = make_pipeline(
    pca_transformer,
    SVC()
)

svm_grid_search_pca = GridSearchCV(pipeline_svm_pca, param_grid, cv=10)
svm_grid_search_pca.fit(train_features, train_labels)
print("Best SVM with PCA Parameters:", svm_grid_search_pca.best_params_)
print("Best SVM with PCA Cross-Validation Accuracy: %.2f%%" % (svm_grid_search_pca.best_score_ * 100))

final_svm_test_accuracy = svm_grid_search_pca.best_estimator_.score(test_features, test_labels)
print("Final SVM Accuracy: %.2f%%" % (final_svm_test_accuracy * 100))

pipeline_sgd = make_pipeline(
    RobustScaler(),
    SGDClassifier()
)
pipeline_sgd.fit(train_features, train_labels)
sgd_test_accuracy = pipeline_sgd.score(test_features, test_labels)
print("SGD Accuracy: %.2f%%" % (sgd_test_accuracy * 100))

pipeline_rf = make_pipeline(
    RobustScaler(),
    RandomForestClassifier()
)
pipeline_rf.fit(train_features, train_labels)
rf_test_accuracy = pipeline_rf.score(test_features, test_labels)
print("RandomForest Accuracy: %.2f%%" % (rf_test_accuracy * 100))

pipeline_mlp = make_pipeline(
    RobustScaler(),
    MLPClassifier()
)
pipeline_mlp.fit(train_features, train_labels)
mlp_test_accuracy = pipeline_mlp.score(test_features, test_labels)
print("MLP Accuracy: %.2f%%" % (mlp_test_accuracy * 100))

accuracy_summary = pd.DataFrame({
    'Model': [
        'SVM',
        'SVM with Hyperparameter Tuning',
        'SVM with Feature Selection',
        'SVM with PCA',
        'SGD',
        'RandomForest',
        'MLP'
    ],
    'Accuracy': [
        svm_test_accuracy,
        svm_grid_search.best_score_,
        svm_grid_search_kbest.best_score_,
        svm_grid_search_pca.best_score_,
        sgd_test_accuracy,
        rf_test_accuracy,
        mlp_test_accuracy
    ]
})

print("\nSummary of Model Accuracies:")
print(accuracy_summary)
